In [10]:
# Set project directory
import os
import sys


def project_dir():
    notebook_path = %pwd
    repo_name = "sisi"
    repo_folder = notebook_path.split(repo_name)[0]
    return os.path.join(repo_folder, repo_name)


pwd = os.getenv("PWD", project_dir())
os.environ["PWD"] = pwd
sys.path.append(pwd)

In [11]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from Utils.data_utils import load_train_test_data
from matplotlib import pyplot as plt
from pandas import Timestamp
import pickle
import warnings
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)

In [12]:
(train_list, test_list), state_list = load_train_test_data(pwd)

In [13]:
test_list[0]

,carbon_per_MWh
Date,
2021-12-31 01:00:00+00:00,412.899235
2021-12-31 02:00:00+00:00,394.983381
2021-12-31 03:00:00+00:00,373.891838
2021-12-31 04:00:00+00:00,343.876998
2021-12-31 05:00:00+00:00,325.939101
...,...
2022-05-12 01:00:00+00:00,284.150513
2022-05-12 02:00:00+00:00,291.982263
2022-05-12 03:00:00+00:00,288.439333


In [14]:
#Creating lag value for each state '''probably not needed'''
for i in range(len(train_list)):
    train_list[i]['lag1'] = train_list[i]['carbon_per_MWh'].shift(24)
    train_list[i] = train_list[i].dropna()
    
for i in range(len(test_list)):
    test_list[i]['lag1'] = test_list[i]['carbon_per_MWh'].shift(24)
    test_list[i] = test_list[i].dropna()

In [15]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [16]:
train_list[0]

,carbon_per_MWh,lag1
Date,,
2018-07-04 05:00:00+00:00,425.727761,428.530509
2018-07-04 06:00:00+00:00,406.542612,430.069745
2018-07-04 07:00:00+00:00,397.211138,486.440186
2018-07-04 08:00:00+00:00,384.970887,512.173077
2018-07-04 09:00:00+00:00,383.051737,510.579051
...,...,...
2021-12-30 20:00:00+00:00,387.267303,308.580147
2021-12-30 21:00:00+00:00,371.341878,320.544101
2021-12-30 22:00:00+00:00,360.994811,322.059430


In [17]:
test_list[0]

,carbon_per_MWh,lag1
Date,,
2022-01-01 01:00:00+00:00,320.516007,412.899235
2022-01-01 02:00:00+00:00,317.448522,394.983381
2022-01-01 03:00:00+00:00,357.694642,373.891838
2022-01-01 04:00:00+00:00,376.555328,343.876998
2022-01-01 05:00:00+00:00,385.025764,325.939101
...,...,...
2022-05-12 01:00:00+00:00,284.150513,287.571776
2022-05-12 02:00:00+00:00,291.982263,296.865662
2022-05-12 03:00:00+00:00,288.439333,302.000181


In [ ]:
predictions = []
errors = []
for train, test in zip(train_list, test_list):
    train_features = np.array(train['lag1'])
    train_labels = np.array(train['carbon_per_MWh'])
    test_features = np.array(test['lag1'])
    test_labels = np.array(test['carbon_per_MWh'])
    rf.fit(train_features.reshape(-1,1), train_labels)
    prediction = rf.predict(test_features.reshape(-1,1))
    predictions.append(prediction)
    errors.append(abs(prediction - test_labels))

In [ ]:
plt.figure(figsize=(16,16))
plt.plot(train_list[0].index, train_list[0]['carbon_per_MWh'], color='black')
plt.plot(test_list[0].index, test_list[0]['carbon_per_MWh'], color='red')
plt.plot(test_list[0].index, predictions[0], color='blue')

In [ ]:
hour_dict_list = []
for i in range(len(test_list)):
    hour_dict = {test_list[i].index[j] : predictions[i][j] for j in range(len(predictions[i]))}
    hour_dict_list.append(hour_dict)

In [ ]:
predictions[0]

In [ ]:
file = open(f'{pwd}/data/predictions/rfpred.pkl', 'wb')
pickle.dump(hour_dict_list, file)
file.close()
file2 = open(f'{pwd}/data/predictions/state_list.pkl', 'wb')
pickle.dump(state_list, file2)
file2.close()